In [2]:
import numpy as np
import pandas as pd

In [3]:
dataset = pd.read_csv("../data/training_data.csv")
dataset = pd.DataFrame(dataset)

In [4]:
dataset

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,619,CASH_IN,386385.08,C421351828,4669568.85,5055953.92,C1977099364,506035.06,119649.98,0.0,0.0
1,164,CASH_IN,212458.78,C83569848,234635.00,447093.78,C1690589535,806037.88,593579.10,0.0,0.0
2,382,PAYMENT,19967.60,C852995095,3634.00,0.00,M1695416333,0.00,0.00,0.0,0.0
3,180,CASH_OUT,527616.51,C61761046,180216.00,0.00,C577654587,92157.10,619773.61,0.0,0.0
4,36,TRANSFER,206067.85,C758004147,0.00,0.00,C2143015292,2131494.48,2337562.32,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
143921,236,CASH_IN,184782.46,C2000187312,1512205.59,1696988.05,C1303211682,293982.59,109200.13,0.0,0.0
143922,347,PAYMENT,31391.33,C1729609363,31152.00,0.00,M602706381,0.00,0.00,0.0,0.0
143923,311,CASH_IN,255521.98,C1034541925,1237508.67,1493030.65,C1088098741,768396.97,512874.99,0.0,0.0
143924,11,CASH_OUT,78284.35,C1847843274,31562.00,0.00,C697452681,2324888.80,2420233.51,0.0,0.0


In [22]:
dataset.columns = dataset.columns.str.strip()  # Removes leading and trailing spaces
print(dataset.isnull().sum()) # Check for missing values
print("---")
print(dataset.duplicated().sum()) # Check for duplicate values


step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64
---
0


In [21]:
dataset.dropna(inplace=True)
dataset.drop_duplicates(inplace=True)

In [7]:
'''
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Assuming your dataset is already loaded, e.g.:
# dataset = pd.read_csv("your_file.csv")

# Select independent features and dependent target
X = dataset[['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']] # Features
Y = dataset['isFraud'] # Target

# Create a new DataFrame that includes both X and Y for plotting
data_with_target = X.copy()  # Copy X to avoid modifying it directly
data_with_target['isFraud'] = Y  # Add the target 'isFraud' to the DataFrame

# Plot pairwise scatter plots for each feature in X against Y
for col in X.columns:
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=col, y='isFraud', data=data_with_target)
    plt.title(f'Scatter plot of {col} vs isFraud')
    plt.show()
'''

'\nimport seaborn as sns\nimport matplotlib.pyplot as plt\nimport pandas as pd\n\n# Assuming your dataset is already loaded, e.g.:\n# dataset = pd.read_csv("your_file.csv")\n\n# Select independent features and dependent target\nX = dataset[[\'amount\',\'oldbalanceOrg\',\'newbalanceOrig\',\'oldbalanceDest\',\'newbalanceDest\']] # Features\nY = dataset[\'isFraud\'] # Target\n\n# Create a new DataFrame that includes both X and Y for plotting\ndata_with_target = X.copy()  # Copy X to avoid modifying it directly\ndata_with_target[\'isFraud\'] = Y  # Add the target \'isFraud\' to the DataFrame\n\n# Plot pairwise scatter plots for each feature in X against Y\nfor col in X.columns:\n    plt.figure(figsize=(6, 4))\n    sns.scatterplot(x=col, y=\'isFraud\', data=data_with_target)\n    plt.title(f\'Scatter plot of {col} vs isFraud\')\n    plt.show()\n'

In [12]:
X = dataset[['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']] # Features (independent)
Y = dataset['isFraud'].values.reshape(-1, 1) # Target (dependent)

In [13]:
print(X.shape)
print(Y.shape)

(143926, 5)
(143926, 1)


In [25]:
X.iloc[1:3]


,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
1,212458.78,234635.0,447093.78,806037.88,593579.1
2,19967.60,3634.0,0.00,0.00,0.0


In [ ]:
def split(rows, qt):
    true_row=[]; false_row=[]

    for i in rows:
        if qt.match(i):
            true_row.append(i)
        else:
            false_row.append(i)
    return true_row,false_row


In [26]:
class qt:
    def _init_(self,col,val):
        self.col = col
        self.val = val
    
    def match(self, ex):
        val = ex[self.col]
        if is_numeric(val):
            return val>= self.val
        else:
            return val == self.val
    
    def __repr__(self):
        condition = "=="
        if is_numeric(self.val):
            condition = ">="
        return "Is %s %s %s ?" % (header[self.col], condition, str(self.val))


In [39]:
dict = dataset['type'].value_counts().to_dict()
for k,v in dict.items():
    print(k,"-->",v)

CASH_OUT --> 50733
PAYMENT --> 48683
CASH_IN --> 31397
TRANSFER --> 12154
DEBIT --> 958


In [55]:
def entropy(col,dataset):
    subset_fraud = dataset[dataset['isFraud']==1.0] # Taking only the rows with isFraud=1
    col_len = len(subset_fraud) # Length of the column
    dict_fraud = subset_fraud[col].value_counts().to_dict() # Making dict of each value in 

    ent=0

    for k,v in dict_fraud.items():
        p = v/col_len
        ent -= p* np.log2(p)

    subset_nfraud = dataset[dataset['isFraud']==0.0]
    col_len = len(subset_nfraud)
    dict_nfraud = subset_fraud[col].value_counts().to_dict()

    for k,v in dict_nfraud.items():
        p = v/col_len
        ent -= p* np.log2(p)


    return ent
    
entropy('type',dataset)

np.float64(1.0140118408314054)

In [43]:
dict = dataset['isFraud'].value_counts().to_dict()
for k,v in dict.items():
    print(k,"-->",v)

subset= dataset[dataset['isFraud']==1.0]
subset

0.0 --> 143733
1.0 --> 192


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
550,61,TRANSFER,1735466.52,C182416415,1735466.52,0.00,C180492809,0.00,0.00,1.0,0.0
636,558,TRANSFER,316877.85,C350145239,316877.85,0.00,C1384673123,0.00,0.00,1.0,0.0
864,142,CASH_OUT,78810.30,C688380113,78810.30,0.00,C1780087922,802788.52,881598.82,1.0,0.0
870,275,CASH_OUT,513082.74,C1294288160,513082.74,0.00,C1140210219,32452.79,545535.53,1.0,0.0
1472,617,TRANSFER,2542664.27,C786455622,2542664.27,2542664.27,C661958277,0.00,0.00,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
141106,219,TRANSFER,555731.98,C2046149869,555731.98,0.00,C1910853781,0.00,0.00,1.0,0.0
141655,658,CASH_OUT,24522.52,C1046356419,24522.52,0.00,C1603948163,155677.01,180199.53,1.0,0.0
142855,483,CASH_OUT,526119.49,C1055498407,526119.49,0.00,C277547037,340510.43,866629.92,1.0,0.0
143256,375,TRANSFER,667733.81,C1449311251,667733.81,0.00,C1975743671,0.00,0.00,1.0,0.0
